In [1]:
import os
import pandas as pd
import numpy as np
import scipy.io
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, 
    Reshape, multiply, add, Activation, Lambda, Concatenate
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

print("TensorFlow Version:", tf.__version__)

2025-09-27 15:37:45.500899: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/Nitin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


TensorFlow Version: 2.16.2


In [ ]:

DATASET_PATH = '~/Downloads/DDMM/deep_ball_raw/BearingType_DeepGrooveBall/SamplingRate_16000'

# Model Hyperparameters
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 30 # Adversarial training can take longer to converge

In [3]:
def parse_filenames_to_dataframe(dataset_path):
    """Walks through the dataset directory and parses filenames."""
    records = []
    expanded_path = os.path.expanduser(dataset_path)
    print(f"Scanning directory: {expanded_path}...")

    if not os.path.isdir(expanded_path):
        print(f"Error: Directory not found. Check DATASET_PATH.")
        return pd.DataFrame()

    for root, _, files in os.walk(expanded_path):
        for file in files:
            if file.endswith('.mat'):
                parts = file.replace('.mat', '').split('_')
                if len(parts) == 5:
                    rotating_fault, bearing_fault, _, _, speed = parts
                    records.append({
                        'filepath': os.path.join(root, file),
                        'filename': file,
                        'speed': int(speed)
                    })
    df = pd.DataFrame(records)
    print(f"Scan complete. Found {len(df)} total .mat files.")
    return df

In [4]:
master_df = parse_filenames_to_dataframe(DATASET_PATH)
if not master_df.empty:
    display(master_df.head())

Scanning directory: /Users/Nitin/Downloads/DDMM/deep_ball_raw/BearingType_DeepGrooveBall/SamplingRate_16000...
Scan complete. Found 192 total .mat files.


,filepath,filename,speed
0,/Users/Nitin/Downloads/DDMM/deep_ball_raw/Bear...,U3_IR_16_6204_600.mat,600
1,/Users/Nitin/Downloads/DDMM/deep_ball_raw/Bear...,M1_H_16_6204_600.mat,600
2,/Users/Nitin/Downloads/DDMM/deep_ball_raw/Bear...,M2_OR_16_6204_600.mat,600
3,/Users/Nitin/Downloads/DDMM/deep_ball_raw/Bear...,M1_OR_16_6204_600.mat,600
4,/Users/Nitin/Downloads/DDMM/deep_ball_raw/Bear...,U1_B_16_6204_600.mat,600


In [5]:
def process_labels(df):
    """Creates binary task labels and domain (speed) labels."""
    print("\nProcessing labels...")
    
    # 1. Binary Task Label (Healthy vs. Faulty)
    # A file is 'Healthy' only if its name is 'H_H_...'
    df['binary_label'] = df['filename'].apply(lambda x: 0 if x.startswith('H_H_') else 1)
    
    # 2. Domain Label (for speed)
    df['domain_label'], domain_classes = pd.factorize(df['speed'])

    print("Label processing complete.")
    print("\nBinary Label Distribution:\n", df['binary_label'].value_counts())
    print("\nDomain (Speed) Classes:", {i: label for i, label in enumerate(domain_classes)})
    
    return df, list(domain_classes)

In [6]:
if not master_df.empty:
    master_df, domain_classes = process_labels(master_df)
    display(master_df[['filename', 'binary_label', 'speed', 'domain_label']].head())


Processing labels...
Label processing complete.

Binary Label Distribution:
 binary_label
1    186
0      6
Name: count, dtype: int64

Domain (Speed) Classes: {0: 600, 1: 1400, 2: 1000, 3: 800, 4: 1600, 5: 1200}


,filename,binary_label,speed,domain_label
0,U3_IR_16_6204_600.mat,1,600,0
1,M1_H_16_6204_600.mat,1,600,0
2,M2_OR_16_6204_600.mat,1,600,0
3,M1_OR_16_6204_600.mat,1,600,0
4,U1_B_16_6204_600.mat,1,600,0


In [ ]:
def load_and_process_file(filepath, binary_label, domain_label, num_domain_classes):
    """Loads a single .mat file and processes its 78 spectrograms."""
    mat_contents = scipy.io.loadmat(filepath.numpy())
    spectrogram_data = mat_contents['Spectrogram']
    
    if np.max(spectrogram_data) > 0:
        spectrogram_data = spectrogram_data / np.max(spectrogram_data)
    images = np.expand_dims(spectrogram_data, axis=-1)
    
    binary_labels = np.array([binary_label] * 78, dtype=np.float32)
    domain_labels_cat = to_categorical([domain_label] * 78, num_classes=num_domain_classes)
    
    return images, binary_labels, domain_labels_cat

def tf_load_and_process(filepath, binary_label, domain_label, num_domain_classes):
    """A TensorFlow wrapper for the loading function."""
    images, bin_labels, dom_labels = tf.py_function(
        func=load_and_process_file,
        inp=[filepath, binary_label, domain_label, num_domain_classes],
        Tout=(tf.float32, tf.float32, tf.float32)
    )
    
    images.set_shape([78, IMG_SIZE, IMG_SIZE, 1])
    bin_labels.set_shape([78])
    dom_labels.set_shape([78, num_domain_classes])
    
    labels_dict = {"task_output": bin_labels, "domain_output": dom_labels}
    return tf.data.Dataset.from_tensor_slices((images, labels_dict))

In [8]:
if not master_df.empty:
    num_domain_classes = len(domain_classes)
    train_df, val_df = train_test_split(master_df, test_size=0.2, random_state=42)

    # --- Build the Training Pipeline ---
    train_dataset = tf.data.Dataset.from_tensor_slices((
        train_df['filepath'].values,
        train_df['binary_label'].values,
        train_df['domain_label'].values
    ))
    train_dataset = train_dataset.shuffle(buffer_size=len(train_df))
    train_dataset = train_dataset.interleave(
        lambda fp, bl, dl: tf_load_and_process(fp, bl, dl, num_domain_classes),
        cycle_length=tf.data.AUTOTUNE, num_parallel_calls=tf.data.AUTOTUNE
    )
    train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # --- Build the Validation Pipeline ---
    val_dataset = tf.data.Dataset.from_tensor_slices((
        val_df['filepath'].values,
        val_df['binary_label'].values,
        val_df['domain_label'].values
    ))
    val_dataset = val_dataset.interleave(lambda fp, bl, dl: tf_load_and_process(fp, bl, dl, num_domain_classes))
    val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    print("\n✅ Full tf.data pipelines created successfully.")


✅ Full tf.data pipelines created successfully.


In [9]:
@tf.custom_gradient
def grad_reverse(x):
    y = tf.identity(x)
    def custom_grad(dy):
        return -dy
    return y, custom_grad

class GradientReversal(tf.keras.layers.Layer):
    """Custom layer to reverse the gradient during backpropagation."""
    def __init__(self):
        super().__init__()

    def call(self, x):
        return grad_reverse(x)

def cbam_module(x, ratio=8):
    """Creates a Convolutional Block Attention Module (CBAM)."""
    # --- Channel Attention ---
    channels = x.shape[-1]
    avg_pool = GlobalAveragePooling2D()(x)
    max_pool = tf.keras.layers.GlobalMaxPooling2D()(x)
    
    shared_dense_one = Dense(channels // ratio, activation='relu')
    shared_dense_two = Dense(channels)
    
    avg_out = shared_dense_two(shared_dense_one(avg_pool))
    max_out = shared_dense_two(shared_dense_one(max_pool))
    
    channel_attention = Activation('sigmoid')(add([avg_out, max_out]))
    channel_attention = Reshape((1, 1, channels))(channel_attention)
    x_channel = multiply([x, channel_attention])

    # --- Spatial Attention ---
    avg_pool = Lambda(lambda y: tf.reduce_mean(y, axis=3, keepdims=True))(x_channel)
    max_pool = Lambda(lambda y: tf.reduce_max(y, axis=3, keepdims=True))(x_channel)
    
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    spatial_attention = Conv2D(1, (7, 7), padding='same', activation='sigmoid')(concat)
    
    return multiply([x_channel, spatial_attention])

In [10]:
def build_full_model(input_shape, num_domain_classes):
    """Builds the Domain-Adaptive CNN with CBAM."""
    inputs = Input(shape=input_shape)

    # --- Step 2: CNN Backbone ---
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    # --- Step 3: Attention Module ---
    attention_output = cbam_module(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(attention_output)
    features = MaxPooling2D((2, 2))(x)
    flattened_features = Flatten()(features)

    # --- Branch 1: Main Task (Binary Classification) ---
    # --- Step 5 (Modified): Binary Classification Head ---
    task_head = Dense(128, activation='relu')(flattened_features)
    task_output = Dense(1, activation='sigmoid', name='task_output')(task_head)

    # --- Branch 2: Domain Adaptation ---
    # --- Step 4: Domain Adaptation Head ---
    grl = GradientReversal()(flattened_features)
    domain_head = Dense(128, activation='relu')(grl)
    domain_output = Dense(num_domain_classes, activation='softmax', name='domain_output')(domain_head)
    
    # --- Create and Compile Model ---
    model = Model(inputs=inputs, outputs=[task_output, domain_output])
    return model

In [11]:
if 'master_df' in locals():
    model = build_full_model(
        input_shape=(IMG_SIZE, IMG_SIZE, 1), 
        num_domain_classes=num_domain_classes
    )

    # Compile the model with two different losses
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'task_output': 'binary_crossentropy',
            'domain_output': 'categorical_crossentropy'
        },
        loss_weights={'task_output': 1.0, 'domain_output': 0.5}, # Weight the domain loss less
        metrics={'task_output': 'accuracy'}
    )
    
    print("\n--- Full Model Summary ---")
    model.summary()


--- Full Model Summary ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ max_pooling2d_1[… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 64)        │          0 │ max_pooling2d_1[… │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 8)         │        520 │ global_average_p… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │        576 │ dense[0][0],      │
│                     │                   │            │ dense[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 64)        │          0 │ dense_1[0][0],    │
│                     │                   │            │ dense_1[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 64)        │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 64)  │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 32, 32,    │          0 │ max_pooling2d_1[… │
│                     │ 64)               │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 32, 32, 1) │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 32, 32, 1) │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32, 2) │          0 │ lambda[0][0],     │
│ (Concatenate)       │                   │            │ lambda_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32, 1) │         99 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 32, 32,    │          0 │ multiply[0][0],   │
│ (Multiply)          │ 64)               │            │ conv2d_2[0][0]  

 Total params: 8,483,634 (32.36 MB)

 Trainable params: 8,483,634 (32.36 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
if 'model' in locals():
    print("\n--- Starting Full Model Training ---")
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        verbose=1
    )
    print("\n--- Model Training Complete ---")


--- Starting Full Model Training ---
Epoch 1/30
    373/Unknown 167s 437ms/step - domain_output_loss: 17.1059 - loss: 221.2057 - task_output_accuracy: 0.9625 - task_output_loss: 212.6528

2025-09-27 15:41:24.889616: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/Nitin/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


373/373 ━━━━━━━━━━━━━━━━━━━━ 181s 474ms/step - domain_output_loss: 17.0836 - loss: 220.9388 - task_output_accuracy: 0.9624 - task_output_loss: 212.3961 - val_domain_output_loss: 1.7784 - val_loss: 1.4414 - val_task_output_accuracy: 0.9744 - val_task_output_loss: 0.5372
Epoch 2/30


2025-09-27 15:41:38.763344: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


373/373 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - domain_output_loss: 1.7907 - loss: 1.4025 - task_output_accuracy: 0.9822 - task_output_loss: 0.5072

...

2025-09-27 17:10:14.878735: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


373/373 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - domain_output_loss: 1.7959 - loss: 1.0577 - task_output_accuracy: 0.9626 - task_output_loss: 0.1598

2025-09-27 17:13:12.181886: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


373/373 ━━━━━━━━━━━━━━━━━━━━ 193s 514ms/step - domain_output_loss: 1.7959 - loss: 1.0577 - task_output_accuracy: 0.9626 - task_output_loss: 0.1597 - val_domain_output_loss: 1.8102 - val_loss: 1.0351 - val_task_output_accuracy: 0.9744 - val_task_output_loss: 0.1185

--- Model Training Complete ---


2025-09-27 17:13:27.903694: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [13]:
if 'history' in locals():
    print("\n--- Evaluating Final Model on Validation Set ---")
    results = model.evaluate(val_dataset)
    
    print("\n--- Final Performance ---")
    print(f"Total Validation Loss: {results[0]:.4f}")
    print(f"Binary Task (Fault/Healthy) Validation Loss: {results[1]:.4f}")
    print(f"Domain (Speed) Validation Loss: {results[2]:.4f}")
    print(f"Binary Task (Fault/Healthy) Validation Accuracy: {results[3]*100:.2f}%")


--- Evaluating Final Model on Validation Set ---
96/96 ━━━━━━━━━━━━━━━━━━━━ 14s 129ms/step - domain_output_loss: 1.8280 - loss: 0.9741 - task_output_accuracy: 0.9925 - task_output_loss: 0.0599

--- Final Performance ---
Total Validation Loss: 1.0351
Binary Task (Fault/Healthy) Validation Loss: 0.1185
Domain (Speed) Validation Loss: 1.8102
Binary Task (Fault/Healthy) Validation Accuracy: 97.44%


2025-09-27 17:24:15.823852: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
